In [1]:
import mysql.connector

remote = mysql.connector.connect(
    host = "database-1.cba44miw26by.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "cwh",
    password = "0000",
    database = "pig"
)

In [2]:
cur = remote.cursor()

In [3]:
# 스마트팜 축산 api key : 최원호
api_key = '97e220f3136949bb9f80dd170ff6dd08'

In [4]:
import pandas as pd

feed_df = pd.DataFrame(columns=['year', 'month', 'statusCode', 'statusMessage', 'farmId', 'sowId', 'matingDt', 'farrowDt', 'mesureDt', 'feedingSetupQy', 'feedingQy'])
feed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            0 non-null      object
 1   month           0 non-null      object
 2   statusCode      0 non-null      object
 3   statusMessage   0 non-null      object
 4   farmId          0 non-null      object
 5   sowId           0 non-null      object
 6   matingDt        0 non-null      object
 7   farrowDt        0 non-null      object
 8   mesureDt        0 non-null      object
 9   feedingSetupQy  0 non-null      object
 10  feedingQy       0 non-null      object
dtypes: object(11)
memory usage: 124.0+ bytes


In [5]:
import urllib.request # urllib = URL 처리를 위한 파이썬 패키지
import json

serviceKey = api_key

def get_feed_breed(year, month, day):
    if month < 10:
        month = f'0{month}'
    else:
        month = str(month)
    if day < 10:
        day = f'0{day}'
    else:
        day = str(day)
    # url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/StockRestService/getMoPigFdBrdDataList/{serviceKey}/{date}/{date}" # 급이 번식
    url = f"http://www.smartfarmkorea.net/Agree_WS/webservices/StockRestService/getMoPigDataList/{serviceKey}/{year}{month}{day}/{year}{month}{day}" # 급이
    request = urllib.request.Request(url) # url 요청
    response = json.loads(urllib.request.urlopen(request).read().decode('utf-8'))
    values_list = list(response[0].values())

    if values_list[2] is None:
        return None

    # None 값을 처리하는 함수
    def safe_int(value, default=0):
        return int(value) if value is not None else default

    def safe_float(value, default=0.0):
        return float(value) if value is not None else default

    def safe_str(value, default=""):
        return str(value) if value is not None else default

    statusCode = safe_str(values_list[0])
    statusMessage = safe_str(values_list[1])
    farmId = safe_str(values_list[2])
    sowId = safe_int(values_list[3])
    matingDt = safe_str(values_list[4])
    farrowDt = safe_str(values_list[5])
    mesureDt = safe_str(values_list[6])
    feedingSetupQy = safe_float(values_list[7])
    feedingQy = safe_float(values_list[8])

    
    return (year, month, statusCode, statusMessage, farmId, sowId, matingDt, farrowDt, mesureDt, feedingSetupQy, feedingQy)
    
    # cur.execute(f"""
    #     INSERT INTO feed_breed VALUES (
    #         {year},
    #         {statusCode}, 
    #         '{statusMessage}', 
    #         '{farmId}', 
    #         {sowId},
    #         '{matingDt}',
    #         '{farrowDt}',
    #         '{mesureDt}',
    #         {feedingSetupQy},
    #         {feedingQy}
    #     )""") # 데이터 삽입

In [6]:
from tqdm import tqdm

for y in tqdm(range(2019, 2025)):
    for i in tqdm(range(1, 13)):
        if (i in [1,3,5,7,8, 10, 12]):
            for j in tqdm(range(1, 32)):
                feed_df.loc[len(feed_df)] = get_feed_breed(y, i, j)
        if (i in [4,6,9,11]):
            for j in tqdm(range(1, 31)):
                feed_df.loc[len(feed_df)] = get_feed_breed(y, i, j)
        if (i in [2]):
            for j in tqdm(range(1, 29)):
                feed_df.loc[len(feed_df)] = get_feed_breed(y, i, j)
    
    
# remote.commit() # 데이터베이스에 반영


  0%|          | 0/6 [00:00<?, ?it/s]































100%|██████████| 31/31 [00:08<00:00,  3.84it/s]





























100%|██████████| 28/28 [00:08<00:00,  3.30it/s]
































100%|██████████| 31/31 [00:09<00:00,  3.39it/s]































100%|██████████| 30/30 [00:08<00:00,  3.48it/s]
































100%|██████████| 31/31 [00:08<00:00,  3.48it/s]































100%|██████████| 30/30 [00:08<00:00,  3.58it/s]
































100%|██████████| 31/31 [00:09<00:00,  3.41it/s]
































100%|██████████| 31/31 [00:08<00:00,  3.57it/s]































100%|██████████| 30/30 [00:09<00:00,  3.25it/s]
































100%|██████████| 31/31 [00:08<00:00,  3.78it/s]































100%|██████████| 30/30 [00:08<00:00,  3.46it/s]
































 17%|█▋        | 1/6 [01:44<08:42, 104.54s/it]













In [7]:
feed_df = feed_df.dropna(subset=['year'])
feed_df.head()

,year,month,statusCode,statusMessage,farmId,sowId,matingDt,farrowDt,mesureDt,feedingSetupQy,feedingQy
355,2019.0,12,00,NORMAL_CODE,PF_0000347_01,1292.0,20190828,20191222,2020-01-16 20:24:11,7.8,7.5
356,2019.0,12,00,NORMAL_CODE,PF_0000347_01,1672.0,20190903,20191223,2020-01-16 20:23:24,7.7,5.2
357,2019.0,12,00,NORMAL_CODE,PF_0020239,2.0,20190902,20191224,2020-01-16 16:27:11,3.5,3.6
358,2019.0,12,00,NORMAL_CODE,PF_0000347_01,292.0,20190903,20191225,2020-01-16 20:23:27,6.9,6.6
360,2019.0,12,00,NORMAL_CODE,PF_0000347_01,104.0,20190903,20191227,2020-01-16 20:23:49,6.3,5.1


In [16]:
feed_df.astype({'year': 'int', 'month': 'int'}).head()

,year,month,statusCode,statusMessage,farmId,sowId,matingDt,farrowDt,mesureDt,feedingSetupQy,feedingQy
355,2019,12,00,NORMAL_CODE,PF_0000347_01,1292.0,20190828,20191222,2020-01-16 20:24:11,7.8,7.5
356,2019,12,00,NORMAL_CODE,PF_0000347_01,1672.0,20190903,20191223,2020-01-16 20:23:24,7.7,5.2
357,2019,12,00,NORMAL_CODE,PF_0020239,2.0,20190902,20191224,2020-01-16 16:27:11,3.5,3.6
358,2019,12,00,NORMAL_CODE,PF_0000347_01,292.0,20190903,20191225,2020-01-16 20:23:27,6.9,6.6
360,2019,12,00,NORMAL_CODE,PF_0000347_01,104.0,20190903,20191227,2020-01-16 20:23:49,6.3,5.1


In [ ]:
feed_df_month = feed_df.pivot_table(index=['year','month'], values=['feedingSetupQy', 'feedingQy'], aggfunc='mean')
feed_df_month.index = feed_df_month.index.set_levels([feed_df_month.index.levels[0].astype(int), feed_df_month.index.levels[1]])
feed_df_month = feed_df_month.round({'feedingQy': 2, 'feedingSetupQy': 2})
feed_df_month

feedingQy  feedingSetupQy
year month                           
2019 12          6.01            6.58
2020 01          3.54            4.24
     02          3.18            3.55
     03          2.91            3.18
     04          4.00            5.74
     05          2.47            3.12
     06          2.30            2.93
     07          2.60            3.10
     08          2.85            3.66
     09          2.30            4.59
     10          3.05            3.80
     11          4.48            6.85
     12          3.98            5.01
2021 01          3.93            6.13
     02          4.89            6.57
     03          4.35            5.68
     04          4.10            5.03
     05          3.56            3.80
     06          4.19            5.38
     07          4.93            6.93
     08          6.38            8.66
     09          4.06            6.28
     10          4.33            6.35
     11          4.26            5.39
     12          2.77            5.56
2022 01          3.86            4.95
     02          1.88            3.22
     03          2.24            3.39
     04          1.77            3.85
     05          1.99            3.73
     06          2.54            4.84
     07          1.37            2.87
     08          2.69            4.16
     09          1.79            4.93
     10          1.56            3.75
     11          2.50            4.77
     12          2.15            4.19
2023 01          2.31            3.61
     02          2.10            3.92
     03          2.88            4.51
     04          2.53            3.51
     05          4.10            6.52
     06          2.96            4.48
     07          3.03            5.70
     08          2.82            4.50
     09          2.99            4.69
     10          2.59            5.21
     11          3.34            5.30
     12          3.92            5.18
2024 01          3.78            4.36
     02          2.47            4.82
     03          2.60            4.69
     04          2.83            4.75
     05          2.25            4.75
     06          2.95            3.85
     07          1.93            2.73

In [25]:
for index, row in feed_df_month.iterrows():
    year = index[0]
    month = index[1]
    feedingSetupQy = row['feedingSetupQy']
    feedingQy = row['feedingQy']
    print(year, month, feedingSetupQy, feedingQy)
    cur.execute(f"""
        INSERT INTO feed_number VALUES (
            {year},
            {month},
            {feedingSetupQy},
            {feedingQy}
        )""") # 데이터 삽입
    
remote.commit() # 데이터베이스에 반영

2019 12 6.58 6.01
2020 01 4.24 3.54
2020 02 3.55 3.18
2020 03 3.18 2.91
2020 04 5.74 4.0
2020 05 3.12 2.47
2020 06 2.93 2.3
2020 07 3.1 2.6
2020 08 3.66 2.85
2020 09 4.59 2.3
2020 10 3.8 3.05
2020 11 6.85 4.48
2020 12 5.01 3.98
2021 01 6.13 3.93
2021 02 6.57 4.89
2021 03 5.68 4.35
2021 04 5.03 4.1
2021 05 3.8 3.56
2021 06 5.38 4.19
2021 07 6.93 4.93
2021 08 8.66 6.38
2021 09 6.28 4.06
2021 10 6.35 4.33
2021 11 5.39 4.26
2021 12 5.56 2.77
2022 01 4.95 3.86
2022 02 3.22 1.88
2022 03 3.39 2.24
2022 04 3.85 1.77
2022 05 3.73 1.99
2022 06 4.84 2.54
2022 07 2.87 1.37
2022 08 4.16 2.69
2022 09 4.93 1.79
2022 10 3.75 1.56
2022 11 4.77 2.5
2022 12 4.19 2.15
2023 01 3.61 2.31
2023 02 3.92 2.1
2023 03 4.51 2.88
2023 04 3.51 2.53
2023 05 6.52 4.1
2023 06 4.48 2.96
2023 07 5.7 3.03
2023 08 4.5 2.82
2023 09 4.69 2.99
2023 10 5.21 2.59
2023 11 5.3 3.34
2023 12 5.18 3.92
2024 01 4.36 3.78
2024 02 4.82 2.47
2024 03 4.69 2.6
2024 04 4.75 2.83
2024 05 4.75 2.25
2024 06 3.85 2.95
2024 07 2.73 1.93


In [26]:
cur.close()
remote.close()